In [ ]:
#Import Packages
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import zscore
from scipy.stats import norm
import os

import holoviews as hv
from colorcet import fire
from sklearn.mixture import GaussianMixture as GMM
from holoviews.operation import gridmatrix
from holoviews.operation.datashader import datashade
from matplotlib.colors import LogNorm
from ssc.cluster import selfrepresentation as sr
import tqdm

hv.extension('bokeh')

# Read in Data and Select Samples

In [ ]:
#Change working directory to where data is stored
os.chdir("/home/idies/workspace/codex/Rare_cell_types/Thymus/gated_SPandVOL_Segmresults_merged")
os.getcwd()

#Read in Dataset and Print Headers
df = pd.read_csv('reg001_comp_SP_merged_real.tsv',sep='\t', header=0)

#Rename Columns
df.columns = df.columns.str.split(':').str[-1].tolist()
df.head()

#Create dataframe to save X, Y columns
df_loc = df.loc[:,['X','Y']]

#Read in clustering dataframe
os.chdir("/home/idies/workspace/codex/Rare_cell_types/Thymus/Xshift_clustering_results")
df_cluster = pd.read_csv('VOLreg1_K10_cl112.csv', header=0)

#Get list of markers that I am subsetting
df_cluster.columns = df_cluster.columns.str.split(':').str[-1].tolist()
df_marker = df_cluster.loc[:,'CD90':'FOXP3']
markerlist = df_marker.columns.tolist()

#Select only the data for working markers
df = df.loc[:,markerlist]

#Select only SampleID you want to examine and only channels for normalization
#array = ['BALBc-3', 'BALBc-2', 'BALBc-1']
#df = df[df.sampleID.isin(array)]

# Perform Transformations

In [ ]:
#Determine which channels to include
first = 0 #Number of channel where markers begin
last = 45 #Last number of channel+1 since non-inclusive
colnum = last-first #Number of total channels

#Select columns with only your data
dfz = df.iloc[:,first:last]

#Save remaining columns in dataframe
dfe = df.iloc[:, np.r_[:first, last:len(df.columns)]]

#zscore of the column markers
dfz1 = pd.DataFrame(zscore(dfz,0),index = dfz.index,columns = ['z0_'+i for i in dfz.columns])

#zscore along rows
dfz2 = pd.DataFrame(zscore(dfz1,1),index = dfz1.index,columns = ['z01_'+i for i in dfz.columns])

#Take cumulative density function to find probability of z score across a row
dfp = pd.DataFrame(norm.cdf(dfz2),index = dfz2.index,columns = ['p_'+i for i in dfz.columns])

#First 1-probability and then take negative logarithm so greater values demonstrate positive cell type
dflog = dfp.apply(lambda x: -np.log(1-x))
dflog.columns = [i for i in dfz.columns]
dflog

In [ ]:
#1 Use universal threshold accross channels
#threshold = 4.6

#2 Component Mixture Model to threshold the plog-values based on 
models = {}
neg_idx = {}
sigma = {}
mu = {}
gmm_thresh = {}

gmm = GMM(n_components=2, n_init=30)

for col in dflog.columns:
    models[col] =  gmm.fit(dflog[col].to_numpy().reshape(-1, 1)) 
    
    neg_idx[col] = np.argmax([models[col].means_])

    sigma[col] = np.sqrt(models[col].covariances_[neg_idx[col]])

    mu[col] = models[col].means_[neg_idx[col],0]
    
    gmm_thresh[col] = mu[col]+1*sigma[col]

In [ ]:
#Replace logarithmic of probability values less than threshold 3 deviations above mean of second mixture model, very little background

dflogcopy = dflog

for col in dflog.columns:
    dflog[col] = (dflog[col] > gmm_thresh[col]).astype(int)

dfr = dflog
dfr.head()

# Finding Unique Cell Types by Thresholding

In [ ]:
#add a unique cell-description to each cell in dataframe and add column with clusterid
dfr['combo'] = dfr.apply(lambda x: '+ '.join(list(x[x == 1].index)), axis=1)
unique = list(dfr['combo'].unique())
dfr['clusterid'] = [unique.index(i) for i in dfr['combo']]

#Concatenate old values to Dataframe
dfcon = pd.concat([dfr,df_loc],1)
dfcon.head()

# Export Data

In [ ]:
#Save dataframe as pickle file
os.chdir("/home/idies/workspace/Storage/jwhickey/persistent")
dfr.to_pickle('dfr_gaus')

#Save interesting columns into csv file
dfexport = dfcon.iloc[:,colnum:len(dfcon.columns)]
dfexport.to_csv('data_tranformed_gaus.csv')

#If you want the whole dataframe
#####dfcon.to_csv('all_data_transformed.csv')

# Exploring Unique Cell Combinations in Python

In [ ]:
#Getting list of column names and then counting unique combinations in dataframe
npcol = dfr.columns.values
nplist = npcol.tolist()
dfcount = dfcon.groupby(nplist).size().to_frame('count').reset_index()

#Sorting the Counts and then thresholding only unique combinations with greater than Cutoff counts
Cutoff = 100
dfsort = dfcount.sort_values(by ='count' , ascending=False).loc[dfcount['count'] >= Cutoff]
dfsort.head(8)

In [ ]:
#Total Cells identified
numcells = len(dfr.index)
identcells = dfsort['count'].sum()
percent = identcells/numcells
print("Total Number of Cells = " +str(numcells),"Total Identified Cells from thresholding = " +str(identcells), 
      "Percent Identified Cells from thresholding = " +str(percent), sep='\n')

In [ ]:
#See top 30 clusterid values of interest
topnum = 30

#Create dataframe for viewing interesting parameters
cluster = dfsort.clusterid.values[:topnum]
combo = dfsort.combo.values[:topnum]
count = dfsort['count'].values[:topnum]
percent_thres = dfsort['count'].values[:topnum]/identcells*100
dftop = pd.DataFrame({'clusterid':cluster,'unique_combo':combo, '%of_ident': percent_thres, 'total_count':count})

#export dataframe for looking at clusters elsewhere
#####dftop.to_csv('top_celltypes.csv')

dftop

# Plots

In [ ]:
#Violin Plots for all of the data
meltz = dflog.melt().sample(n=5000)

#Set Plotting Style and Plot values as violin plots
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)}, font_scale=3.5)
sns.catplot(data = meltz,col= 'variable',height = 15,y = 'value',kind = 'violin',
            col_wrap =5,sharey = False,alpha = .9)

In [ ]:
#Plot the unique combinations as heatmap

#Change the style of the plot
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)}, font_scale=1.0)

#Change the dataframe to be averages by clusterID and do not plot more than number of columns
sns.heatmap(dfsort.iloc[:,:colnum])